### News

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# fourbit_models = [
#     # "unsloth/Llama-3.2-3B-bnb-4bit",
#     "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
#     # "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
# ]

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# !pip install jsonlines
import pandas as pd
file_path = '/content/drive/MyDrive/nlp/subtaskA_train_monolingual.jsonl'

train_df = pd.read_json(file_path, lines = True)
test_df = pd.read_json('/content/drive/MyDrive/nlp/SemEval 2024 Task 8 Monolingual.jsonl', lines = True)
train_df.head()


                                                text  label    model   source  \
0  Forza Motorsport is a popular racing game that...      1  chatGPT  wikihow   
1  Buying Virtual Console games for your Nintendo...      1  chatGPT  wikihow   
2  Windows NT 4.0 was a popular operating system ...      1  chatGPT  wikihow   
3  How to Make Perfume\n\nPerfume is a great way ...      1  chatGPT  wikihow   
4  How to Convert Song Lyrics to a Song'\n\nConve...      1  chatGPT  wikihow   

   id  
0   0  
1   1  
2   2  
3   3  
4   4  


In [8]:
test_df.head()

,text,id
0,"Today, many adults or teenage drivers are hook...",0
1,"The automobile, since its advent, has revoluti...",1
2,One policy that could potentially improve aca...,2
3,Title: Navigating the Road Ahead: The Case for...,3
4,Have you ever woken up in the morning and wish...,4


In [10]:
example_with_label_0 = train_df[train_df['label'] == 0].iloc[0]
example_with_label_0["text"]


" If you're a photographer, keep all the necessary lens, cords, and batteries in the same quadrant of your home or studio. Paints should be kept with brushes, cleaner, and canvas, print supplies should be by the ink, etc. Make broader groups and areas for your supplies to make finding them easier, limiting your search to a much smaller area. Some ideas include:\n\n\nEssential supplies area -- the things you use every day.\nInspiration and reference area.\nDedicated work area .\nInfrequent or secondary supplies area, tucked out of the way.;\n, This doesn't mean cleaning the entire studio, it just means keeping the area immediately around the desk, easel, pottery wheel, etc. clean each night. Discard trash or unnecessary materials and wipe down dirty surfaces. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.\n\n\nEven if the rest of your studio is a bit disorganized, an organized workspace

In [9]:
from datasets import Dataset
def convert_to_conversations(df):
    conversations = []

    for _, row in df.iterrows():
        user_prompt = f"Please determine the origin of the following text and respond with 'LLM' if it is generated by a language model or 'Human' if it is written by a person. Provide your answer in the format: 'Classification: [LLM/Human]'.\n\nText:\n{row['text']}\n\nClassification:"


        # Convert label to string
        label_text = "LLM" if row["label"] == 1 else "Human"

        conversation = [
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": label_text}
        ]
        conversations.append({"conversations": conversation})

    return Dataset.from_list(conversations)

In [10]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
dataset = convert_to_conversations(train_df)
dataset = standardize_sharegpt(dataset)

# Apply formatting
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        ) for convo in convos
    ]
    return { "text" : texts }

dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/119757 [00:00<?, ? examples/s]

Map:   0%|          | 0/119757 [00:00<?, ? examples/s]

In [11]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 119757
})

In [12]:
dataset[5]["conversations"]

[{'content': 'Please determine the origin of the following text and respond with \'LLM\' if it is generated by a language model or \'Human\' if it is written by a person. Provide your answer in the format: \'Classification: [LLM/Human]\'.\n\nText:\nHow to Fix a Broken Window in a Wooden Frame\n\nIf you have a broken window in a wooden frame, don\'t panic! Fixing it is easier than you might think. By following a few simple steps, you can have your window looking like new in no time.\n\nStep 1: Determine what kind of window or door you\'re dealing with\n\nBefore you start fixing the window or door, you need to determine what kind of glass you\'ll need to use. Check your local building codes to see if tempered glass is required for storm windows and doors. If not, you may be able to use regular pane glass for inside (not storm) windows and doors.\n\nStep 2: Remove the window from the casing if possible\n\nIf the broken window is in a wooden frame, you may be able to remove the whole windo

In [13]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease determine the origin of the following text and respond with \'LLM\' if it is generated by a language model or \'Human\' if it is written by a person. Provide your answer in the format: \'Classification: [LLM/Human]\'.\n\nText:\nHow to Fix a Broken Window in a Wooden Frame\n\nIf you have a broken window in a wooden frame, don\'t panic! Fixing it is easier than you might think. By following a few simple steps, you can have your window looking like new in no time.\n\nStep 1: Determine what kind of window or door you\'re dealing with\n\nBefore you start fixing the window or door, you need to determine what kind of glass you\'ll need to use. Check your local building codes to see if tempered glass is required for storm windows and doors. If not, you may be able to use regular pane glass for inside (not

<a name="Train"></a>
### Train the model
 Huggingface TRL's `SFTTrainer`

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"]:   0%|          | 0/119757 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

We verify masking is actually done:

In [15]:
trainer.train_dataset

Dataset({
    features: ['conversations', 'text', 'input_ids', 'attention_mask'],
    num_rows: 119757
})

In [18]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease determine the origin of the following text and respond with \'LLM\' if it is generated by a language model or \'Human\' if it is written by a person. Provide your answer in the format: \'Classification: [LLM/Human]\'.\n\nText:\nHow to Fix a Broken Window in a Wooden Frame\n\nIf you have a broken window in a wooden frame, don\'t panic! Fixing it is easier than you might think. By following a few simple steps, you can have your window looking like new in no time.\n\nStep 1: Determine what kind of window or door you\'re dealing with\n\nBefore you start fixing the window or door, you need to determine what kind of glass you\'ll need to use. Check your local building codes to see if tempered glass is required for storm windows and doors. If not, you may be able to use regular pane glass for inside (not

In [19]:
# space = tokenizer(" ", add_special_tokens = False).input_ids[0]
# tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

KeyError: 'labels'

We can see the System and Instruction prompts are successfully masked!

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.205 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 119,757 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.803000
2,2.580500
3,2.666500
4,3.074000
5,2.373100
6,2.614700
7,2.449800
8,2.557100
9,2.198200
10,2.436800


In [19]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

281.1294 seconds used for training.
4.69 minutes used for training.
Peak reserved memory = 3.938 GB.
Peak reserved memory for training = 1.733 GB.
Peak reserved memory % of max memory = 26.715 %.
Peak reserved memory for training % of max memory = 11.756 %.


In [ ]:
model.save_pretrained("/content/drive/MyDrive/nlp/llama3b_lora_model")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/nlp/llama3b_lora_model")
model.push_to_hub("itsdevansh/llama3b_finetuned_for_llm", token = "") # Online saving
tokenizer.push_to_hub("itsdevansh/llama3b_finetuned_for_llm", token = "") # Online saving

README.md:   0%|          | 0.00/599 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/itsdevansh/llama3b_finetuned_for_llm


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Inference

In [8]:
import os
import torch
import json
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/nlp/llama3b_lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = None,
    )
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [13]:
# === Input text ===
text = """I am an LLM writing this text"""

# === Build prompt ===
prompt_text = (
    "Please determine the origin of the following text and respond with 'LLM' if it is "
    "generated by a language model or 'Human' if it is written by a person. Provide your "
    "answer in the format: 'Classification: [LLM/Human]'.\n\nText:\n"
    f"{text}\n\nClassification:"
)

messages = [{"role": "user", "content": prompt_text}]

# === Tokenize input ===
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

# === Generate prediction ===
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=4,
        temperature=0.1,
        top_p=0.9,
    )

# === Decode output ===
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("\n🔍 Model Output:\n", output_text)

response = output_text.split("Classification:assistant")[-1].strip()

# === Extract prediction from response ===
if "LLM" in response:
    label = "LLM"
elif "Human" in response:
    label = "Human"
else:
    label = "Unknown"

print("\nPredicted Label:", label)



🔍 Model Output:
 system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Please determine the origin of the following text and respond with 'LLM' if it is generated by a language model or 'Human' if it is written by a person. Provide your answer in the format: 'Classification: [LLM/Human]'.

Text:
I am an LLM writing this text

Classification:assistant

LLM

Predicted Label: LLM


In [26]:
# === Output file path ===
output_file = "/content/drive/MyDrive/nlp/predictions_llama.jsonl"

# === Load completed IDs if output file exists ===
completed_ids = set()
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        for line in f:
            if line.strip():  # ensure non-empty
                record = json.loads(line)
                completed_ids.add(record["id"])

# === Run inference on each row ===
# Open the file in append mode to continue from previous predictions.
with open(output_file, "a") as f:
    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        # Skip if this row was already processed
        if row["id"] in completed_ids:
            continue

        # Build the prompt
        prompt_text = (
            f"Please determine the origin of the following text and respond with 'LLM' if it is "
            f"generated by a language model or 'Human' if it is written by a person. Provide your "
            f"answer in the format: 'Classification: [LLM/Human]'.\n\nText:\n{row['text']}\n\nClassification:"
        )
        messages = [{"role": "user", "content": prompt_text}]

        # Tokenize using chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(model.device)

        # Generate output
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=4,
                use_cache=True,
                temperature=0.1,
                top_p=0.9,
            )

        # Decode output
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        response = decoded.split("Classification:assistant")[-1].strip()

        # === Extract prediction from response ===
        if "LLM" in response:
            label = "LLM"
        elif "Human" in response:
            label = "Human"
        else:
            label = "Unknown"

        # Write prediction to .jsonl
        f.write(json.dumps({"id": row["id"], "prediction": label}) + "\n")
        # Optionally flush to ensure the progress is saved immediately.
        f.flush()


100%|██████████| 34272/34272 [3:18:35<00:00,  2.88it/s]


In [27]:
predictions_df = pd.read_json(output_file, lines = True)